In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
import numpy as np
X, y = mnist["data"], mnist["target"]

In [3]:
X[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [4]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [5]:
import numpy as np

def move_image_1px(X, dir):
    image = np.array(X).reshape(28, 28)
    shifted_image = np.zeros((28, 28))

    if dir == "right":
        part = image[:, :-1] 
        shifted_image[:, 1:] = part
        
    elif dir == "left":
        part = image[:, 1:]
        shifted_image[:, :-1] = part
        
    elif dir == "up":
        part = image[1:, :]
        shifted_image[:-1, :] = part

    elif dir == "down":
        part = image[:-1, :]
        shifted_image[1:, :] = part
        
    # Retorna a imagem deslocada (achatada para o formato original de 784)
    return shifted_image.flatten()

In [6]:
X_augmented = []
y_augmented = []
for i in range(0, len(X_train)):
    x_up = move_image_1px(X_train[i], "up")
    x_down = move_image_1px(X_train[i], "down")
    x_left = move_image_1px(X_train[i], "left")
    x_right = move_image_1px(X_train[i], "right")
    
    X_augmented.extend([x_up, x_down, x_left, x_right, X_train[i]])
    
    for i in range(0,5):
        y_augmented.append(y_train[i])
    

In [7]:
print(len(X_augmented), len(X_train), len(y_augmented))


300000 60000 300000


In [8]:
from sklearn.preprocessing import StandardScaler

X_aug_prep = StandardScaler().fit_transform(X_augmented)
X_prep = StandardScaler().fit_transform(X_train)

In [20]:
X_test_aug_prep = StandardScaler().fit(X_augmented).transform(X_test)
X_test_prep = StandardScaler().fit(X_train).transform(X_test)

In [ ]:
from sklearn.linear_model import SGDClassifier

svc = SGDClassifier(loss='hinge', penalty='l2', max_iter=1000, tol=1e-3, random_state=42)
svc.fit(X_aug_prep,y_augmented)


svc.fit(X_prep,y_train)


In [24]:
y_aug_pred = svc.predict(X_test_aug_prep)
y_pred = svc.predict(X_test_prep)

10000

In [31]:
from sklearn.metrics import accuracy_score, f1_score

print("Modelo Normal -- Acuracia: ", accuracy_score(y_test, y_pred), " -- f1: ", f1_score(y_test, y_pred, average='macro'))
print("Modelo Aumentado -- Acuracia: ", accuracy_score(y_test, y_aug_pred), " -- f1: ", f1_score(y_test,y_aug_pred, average="macro"))

Modelo Normal -- Acuracia:  0.8933  -- f1:  0.8959121328654819
Modelo Aumentado -- Acuracia:  0.8868  -- f1:  0.8910498942973891
